<a href="https://colab.research.google.com/github/EdwardFang09/IEE4912/blob/main/dataset_maker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
!huggingface-cli login

Train-test dataset

In [ ]:
import os
from datasets import Dataset, Audio, DatasetDict
import pandas as pd
import re
import librosa

# Define the folder path
folder_path = "./"  # Replace with the actual folder path

# Define the words and their corresponding patterns
words = {
    "sora open browser": r"[bB]rowser \((\d+)\)\.mp3",
    "sora open chrome": r"[cC]hrome \((\d+)\)\.mp3",
    "sora end class": r"[eE]nd [cC]lass \((\d+)\)\.mp3",
    "sora open google": r"[gG]oogle \((\d+)\)\.mp3",
    "sora open kahoot": r"[kK]ahoot \((\d+)\)\.mp3",
    "sora open mentimeter": r"[mM]entimeter \((\d+)\)\.mp3",
    "sora": r"[sS]ora \((\d+)\)\.mp3",
    "sora start class": r"[sS]tart [cC]lass \((\d+)\)\.mp3",
}

# Create a list of audio file paths and sentences
audio_paths = []
sentences = []

for filename in os.listdir(folder_path):
    for word, pattern in words.items():
        match = re.match(pattern, filename)
        if match:
            file_path = os.path.join(folder_path, filename)
            audio_paths.append(file_path)
            sentences.append(word)
            break  # Move to the next filename if a match is found

# Create the dataset
data = {
    "audio": audio_paths,
    "sentence": sentences,
}
df = pd.DataFrame(data)
dataset = Dataset.from_pandas(df)
dataset = dataset.cast_column("audio", Audio())

# Split the dataset into train and test sets
dataset = dataset.train_test_split(test_size=0.2, seed=42)

# Create a DatasetDict
dataset = DatasetDict({
    'train': dataset['train'],
    'test': dataset['test']
})

print(dataset)  # To see the output, run the code.

# Push the dataset to the Hugging Face Hub
dataset.push_to_hub("EdwardFang09/IEE4912_Dataset_v2")

Benchmark dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("EdwardFang09/IEE4912_Dataset_v2")

In [ ]:
import os
from datasets import Dataset, Audio, DatasetDict
import pandas as pd
import re
import librosa

# Define the folder path
folder_path = "./"  # Replace with the actual folder path

# Define the words and their corresponding patterns
words = {
    "sora open chrome": r"chrome.mp3",
    "sora open kahoot": r"kahoot.mp3",
    "sora open mentimeter": r"mentimeter.mp3",
}

# Create a list of audio file paths and sentences
audio_paths = []
sentences = []

for filename in os.listdir(folder_path):
    for word, pattern in words.items():
        match = re.match(pattern, filename)
        if match:
            file_path = os.path.join(folder_path, filename)
            audio_paths.append(file_path)
            sentences.append(word)
            break  # Move to the next filename if a match is found

# Create the benchmark dataset
data = {
    "audio": audio_paths,
    "sentence": sentences,
}
df = pd.DataFrame(data)
benchmark_dataset = Dataset.from_pandas(df)
benchmark_dataset = benchmark_dataset.cast_column("audio", Audio())

In [ ]:
dataset["benchmark"] = benchmark_dataset

In [ ]:
dataset.push_to_hub("EdwardFang09/IEE4912_Dataset_v2")